# 자연어 처리를 통한 성격 예측 : 모델링

In [2]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
warnings.filterwarnings('ignore')               # Turn the warnings off.
from xgboost import XGBClassifier
import joblib

## 학습용 데이터셋 준비

### 1. X_data 준비

#### 1) 말뭉치 데이터 준비

In [7]:
# 데이터 불러오기 & 확인
df = pd.read_csv('mbti_new.csv')
df.head()

,type,posts
0,INFJ,enfp intj moments sportscenter top ten plays p...
1,ENTP,finding lack posts alarming sex boring positio...
2,INTP,good one course say know blessing curse absolu...
3,INTJ,dear intp enjoyed conversation day esoteric ga...
4,ENTJ,fired another silly misconception approaching ...


In [8]:
# 학습을 위한 X데이터 (게시글 데이터) 만들기
# 이를 위해 df의 posts 부분을 리스트 형태의 corpus 객체에 저장해준다.
corpus = []
for post in df.posts:
    corpus.append(post)

In [16]:
# cocpus 객체에 재대로 저장되었나 확인
print("특정 게시글 내용 확인 : " + corpus[500])
print("샘플 수 : " + str(len(corpus)))

특정 게시글 내용 확인 : intj direct disregard society entirely others see importance putting mask online test exactly posses reliability personalities fixed may posses traits one personality yet one another sometimes get intp well number keep home watching hilarious video movies always works fiction emotional entertainment non fiction intellectual entertainment congratulations another day closer death laughing considered old man nothing older intj philosophical inclination surprise really boils definition definition determined society agree damaged core controlled society emotional human related problems psychological problems iii existential depression despair love see problem solving skills come handy use standard model unless given get really worked bed anxious absolutely correct complexity translation tremendously difficult ipo input process output machine input process efficiency pretty normal psychologically society sensitive bunch say something norm get bashed cookie jar comprehend fits 

#### 2) TFIDF 행렬화

In [28]:
# Tfidfvectorizer's parameters 최적화 진행
# 함수 만들기 - 파라미터 별 정확도 파악

def set_param(my_max_features, my_min_df, my_max_df):
    vectorizer = TfidfVectorizer(max_features = my_max_features, min_df = my_min_df, max_df = my_max_df)
    X = vectorizer.fit_transform(corpus).toarray()
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 1234)
    LR = LogisticRegression()
    LR.fit(X_train, Y_train)
    Y_pred = LR.predict(X_test)
    acc = accuracy_score(Y_test, Y_pred)
    
    return acc

In [30]:
# for문과 위에서 만든 함수를 이용하여 정확도가 가장 높은 파라미터 값을 찾는다. 

for i in [500, 1000, 1500]:
    for j in np.arange(1, 5, 1):
        for k in np.arange(0.1, 1.0, 0.1):
            print("max_features가 {}이고 min_df가 {}이며 max_df가 {}일 때의 정확도는 {}입니다.".format(i, j, k, round(set_param(i, j, k),2)))

max_features가 500이고 min_df가 1이며 max_df가 0.1일 때의 정확도는 0.76입니다.
max_features가 500이고 min_df가 1이며 max_df가 0.2일 때의 정확도는 0.79입니다.
max_features가 500이고 min_df가 1이며 max_df가 0.30000000000000004일 때의 정확도는 0.8입니다.
max_features가 500이고 min_df가 1이며 max_df가 0.4일 때의 정확도는 0.83입니다.
max_features가 500이고 min_df가 1이며 max_df가 0.5일 때의 정확도는 0.84입니다.
max_features가 500이고 min_df가 1이며 max_df가 0.6일 때의 정확도는 0.84입니다.
max_features가 500이고 min_df가 1이며 max_df가 0.7000000000000001일 때의 정확도는 0.85입니다.
max_features가 500이고 min_df가 1이며 max_df가 0.8일 때의 정확도는 0.85입니다.
max_features가 500이고 min_df가 1이며 max_df가 0.9일 때의 정확도는 0.85입니다.
max_features가 500이고 min_df가 2이며 max_df가 0.1일 때의 정확도는 0.76입니다.
max_features가 500이고 min_df가 2이며 max_df가 0.2일 때의 정확도는 0.79입니다.
max_features가 500이고 min_df가 2이며 max_df가 0.30000000000000004일 때의 정확도는 0.8입니다.
max_features가 500이고 min_df가 2이며 max_df가 0.4일 때의 정확도는 0.83입니다.
max_features가 500이고 min_df가 2이며 max_df가 0.5일 때의 정확도는 0.84입니다.
max_features가 500이고 min_df가 2이며 max_df가 0.6일 때의 정확도는 0.84입니다.
max_features가 500이고 min_d

In [ ]:
# max feature값이 증가한다고 해서 정확도가 유의미하게 증가하지 않는다. 
# max_feature = 500, min_df = 1, max_df = 0.5가 최적화 파라미터 값이다. 

In [31]:
# 학습을 위해 Tfidfvectorizer 진행
# 위의 최적화를 바탕으로 파라미터값 설정

vectorizer = TfidfVectorizer(max_features = 500, min_df = 1, max_df =0.5)
X = vectorizer.fit_transform(corpus).toarray() 

In [32]:
# X 확인
# 관측치 x 단어집합 크기

X.shape

(8674, 500)

### 2. Y_data(label) 준비

In [ ]:
# Y 데이터 만들기

type = df['type']

# 내/외향성으로 구분 & 0,1로 변환
Y = type.apply(lambda x : x[0])
Y = Y.replace(['I', 'E'], [0, 1])

### 3. data 분리 및 확인

In [21]:
# 데이터 셋 나누기

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 1234)

In [22]:
# 각 데이터셋의 크기 확인

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(6072, 500)
(2603, 500)
(6072,)
(2603,)


## 모델 학습 & 평가

### 1. Rogistic 회귀

In [26]:
# 모델 생성 및 학습

LR = LogisticRegression()
LR.fit(X_train, Y_train)

LogisticRegression()

In [30]:
# 정확도 확인

Y_pred = LR.predict(X_test)
conf_mat = confusion_matrix(Y_test, Y_pred)
acc = accuracy_score(Y_test, Y_pred)
print( "Rogistic accuracy : " + str(np.round(accuracy_score(Y_test,Y_pred),3)))

Rogistic accuracy : 0.851


In [ ]:
# 학습된 모델 저장

joblib.dump(LR, 'LR.pkl')

### 2. XGBoost

In [ ]:
# 모델 생성 및 하이퍼파라미터 최적화

max_depth_grid = np.arange(5, 10, 2)
learning_rate_grid = np.arange(0.02, 0.12, 0.02)                      
n_estimators_grid = np.arange(100, 500, 100)
parameters = {'max_depth': max_depth_grid, 'learning_rate': learning_rate_grid, 'n_estimators': n_estimators_grid}

gridCV = GridSearchCV(XGBClassifier(), parameters, cv=10, n_jobs = -1)
gridCV.fit(X_train, Y_train)

In [173]:
# 정확도 확인

XGBC_best = gridCV.best_estimator_
Y_pred = XGBC_best.predict(X_test)
print( "정확도: " + str(np.round(accuracy_score(Y_test,Y_pred),3)))

XGBoost accuracy : 0.863


In [ ]:
# 학습된 모델 저장 

joblib.dump(XGBC_best, 'XGBC.pkl')

### 3. RandomForest

In [12]:
# 모델 생성 및 하이퍼파라미터 최적화

n_estimators_grid = np.arange(100, 350, 50)
depth_grid = np.arange(5, 30, 3)
min_samples_leaf_grid = np.arange(5,20,4)
min_samples_split_grid = np.arange(8, 21, 4)
parameters = {'n_estimators': n_estimators_grid, 'max_depth': depth_grid, 'min_samples_leaf':min_samples_leaf_grid,
              'min_samples_split':min_samples_split_grid}

gridCV = GridSearchCV(RandomForestClassifier(), parameters, cv=10, n_jobs=-1)
gridCV.fit(X_train, Y_train)

In [14]:
# 정확도 확인

RF_best = gridCV.best_estimator_
RF_best.fit(X_train, Y_train)
Y_pred = RF_best.predict(X_test)
print( "Random Forest best accuracy : " + str(np.round(metrics.accuracy_score(Y_test,Y_pred),3)))

Random Forest best accuracy : 0.802


In [85]:
# 학습된 모델 저장 

joblib.dump(RF_best, 'RF.pkl')

['RF.pkl']

In [33]:
# XGBoost 모델의 정확도가 0.863으로 가장 높다. 